In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from matplotlib import image
import os

In [2]:
#loading 240 images for training & 160 for testing

train_image_list=[]
test_image_list=[]
for i in range(1,41):
    j=1
    s = os.listdir('s'+str(i)+'/')
    for image in s:
        if(j<=6):
            im = Image.open('s'+str(i)+'/' + image)
            train_image_list.append(np.asarray(im).flatten())
        else:
            im = Image.open('s'+str(i)+'/' + image)
            test_image_list.append(np.asarray(im).flatten())
        j+=1

FileNotFoundError: [Errno 2] No such file or directory: 's1/'